In [1]:
import chromedriver_autoinstaller
import time
import xml.etree.ElementTree as ET

import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.firefox import GeckoDriverManager

/home/dell/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [44]:
def pubchem_extractor(cas_numbers):

    if isinstance(cas_numbers, str):
        cas_numbers = [cas_numbers]

    # Instalar o ChromeDriver automaticamente
    chromedriver_autoinstaller.install()
    # driver = get_driver()

    # Inicializar o DataFrame final
    all_data = pd.DataFrame()

    for cas_number in cas_numbers:
        chrome_options = Options()
        #firefoxOptions = Options()
        #firefoxOptions.add_argument("--headless")
        #service = Service(GeckoDriverManager().install())
        #driver = webdriver.Firefox(
        #    options=firefoxOptions,
        #    service=service,)
        driver = webdriver.Chrome(options=chrome_options)
        try:
            # Inicializar o navegador

            # Acessar a página do PubChem
            url = "https://pubchem.ncbi.nlm.nih.gov/"
            driver.get(url)

            time.sleep(3)

            # Inserir o número CAS na barra de pesquisa
            search = driver.find_element(
                By.XPATH,
                "/html/body/div[1]/div/div/main/div[1]/div/div[2]/div/div[2]/form/div/div[1]/input"
            )
            search.send_keys(cas_number)
            search.send_keys(Keys.RETURN)

             # Aguardar até que o elemento clicável esteja disponível
            WebDriverWait(driver, 12).until(
                EC.element_to_be_clickable(
                    (
                        By.XPATH,
                        "/html/body/div[1]/div/div/main/div[2]/div[2]/div[3]/div/div/div/div[2]/ul/li/div/div/div[1]/div[2]/div[1]/a/span/span"
                    )
                )
            ).click()

            # Esperar para carregar os detalhes
            time.sleep(10)

             # Função auxiliar para buscar um elemento de forma segura
            def get_text_or_default(xpath, default="Não encontrado"):
                elements = driver.find_elements(By.XPATH, xpath)
                return elements[0].text if elements else default

            # Extração dos dados
            cid = get_text_or_default('//div[text()="PubChem CID"]/following-sibling::div')
            molecular_formula = get_text_or_default('//div[text()="Molecular Formula"]/following-sibling::div')
            synonyms = get_text_or_default('//div[text()="Synonyms"]/following-sibling::div')
            molecular_weight = get_text_or_default('//div[text()="Molecular Weight"]/following-sibling::div')
            dates = get_text_or_default('//div[text()="Dates"]/following-sibling::div')
            description = get_text_or_default('//div[text()="Description"]/following-sibling::div')

            # Criar um dicionário com os dados extraídos
            dict_data = {
                "CAS Number": [cas_number],
                "CID": [cid],
                "Fórmula Molecular": [molecular_formula],
                "Sinônimos": [synonyms],
                "Peso Molecular": [molecular_weight],
                "Datas": [dates],
                "Descrição": [description]
            }

            # Criar um DataFrame e adicionar ao DataFrame final
            data = pd.DataFrame(dict_data)
            all_data = pd.concat([all_data, data], ignore_index=True)

        except Exception as e:
            print(f"Erro ao processar o CAS Number {cas_number}: {e}")
        finally:
            # Fechar o navegador
            driver.quit()

    return all_data

In [45]:
pubchem_extractor('15663-27-1')

Erro ao processar o CAS Number 15663-27-1: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=125.0.6422.141)
Stacktrace:
#0 0x630c9eabbe3a <unknown>
#1 0x630c9e7a545c <unknown>
#2 0x630c9e77aa84 <unknown>
#3 0x630c9e81c95d <unknown>
#4 0x630c9e832c76 <unknown>
#5 0x630c9e814253 <unknown>
#6 0x630c9e7e41c7 <unknown>
#7 0x630c9e7e4b3e <unknown>
#8 0x630c9ea8227b <unknown>
#9 0x630c9ea86327 <unknown>
#10 0x630c9ea6edae <unknown>
#11 0x630c9ea86df2 <unknown>
#12 0x630c9ea5374f <unknown>
#13 0x630c9eaab128 <unknown>
#14 0x630c9eaab2fb <unknown>
#15 0x630c9eabaf6c <unknown>
#16 0x7e307fa94ac3 <unknown>



""


In [7]:
pubchem_extractor('50-00-0')

Erro ao processar o CAS Number 50-00-0: Message: 
Stacktrace:
#0 0x647ae8b24e3a <unknown>
#1 0x647ae880e45c <unknown>
#2 0x647ae885a5b5 <unknown>
#3 0x647ae885a671 <unknown>
#4 0x647ae889ef14 <unknown>
#5 0x647ae887d4dd <unknown>
#6 0x647ae889c2cc <unknown>
#7 0x647ae887d253 <unknown>
#8 0x647ae884d1c7 <unknown>
#9 0x647ae884db3e <unknown>
#10 0x647ae8aeb27b <unknown>
#11 0x647ae8aef327 <unknown>
#12 0x647ae8ad7dae <unknown>
#13 0x647ae8aefdf2 <unknown>
#14 0x647ae8abc74f <unknown>
#15 0x647ae8b14128 <unknown>
#16 0x647ae8b142fb <unknown>
#17 0x647ae8b23f6c <unknown>
#18 0x7d2af7c94ac3 <unknown>



""


In [52]:
def echa_extractor(cas_numbers):
    # Certificar-se de que cas_numbers é uma lista
    if isinstance(cas_numbers, str):
        cas_numbers = [cas_numbers]

        chromedriver_autoinstaller.install()
    
    # Instalar e configurar o driver automaticamente
    all_data = pd.DataFrame()

    for cas_number in cas_numbers:
        chrome_options = Options()
        #firefoxOptions = Options()
        #firefoxOptions.add_argument("--headless")
        #service = Service(GeckoDriverManager().install())
        #driver = webdriver.Firefox(
            #options=firefoxOptions,
            #service=service,)
        driver = webdriver.Chrome(options=chrome_options)
        try:
            # Acessar a página
            driver.get("https://echa.europa.eu/pt/information-on-chemicals")
            
            # Configurar WebDriverWait
            wait = WebDriverWait(driver, 10)
            actions = ActionChains(driver)
            
            # Aceitar cookies
            cookie_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="cookie-consent-banner"]/div/div/div[2]/a[1]')))
            actions.move_to_element(cookie_button).click().perform()
            
            time.sleep(2)
            
            # Selecionar checkbox
            checkbox = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="_disssimplesearchhomepage_WAR_disssearchportlet_fm"]/div[2]/label/span')))
            actions.move_to_element(checkbox).click().perform()
            
            time.sleep(2)
            
            # Inserir o número CAS na barra de pesquisa
            search = driver.find_element(By.XPATH, '//*[@id="autocompleteKeywordInput"]')
            search.send_keys(cas_number)
            search.send_keys(Keys.RETURN)
            
            # Aguardar até que o elemento esteja visível e clicável
            element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="_disssimplesearch_WAR_disssearchportlet_rmlSearchResultVOsSearchContainerSearchContainer"]/table/tbody/tr[1]/td[1]/a'))
            )
            
            # Clica no elemento
            element.click()
            time.sleep(5)
           # Botão que pode estar ou não desabilitado
            try:
                registered_button = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="_disssubsinfo_WAR_disssubsinfoportlet_dataset-reg-button"]/span')))
                if not registered_button.is_enabled():
                    print(f"Botão 'Registered' desabilitado para o CAS {cas_number}.")
                    continue  # Cenário 3: Botão desabilitado
            except:
                print(f"Botão 'Registered' não encontrado para o CAS {cas_number}.")
                continue

            # Se o botão "Registered" está habilitado, verificar se abre uma nova aba
            registered_button.click()
            time.sleep(2)

            # Verificar se uma nova aba foi aberta
            tabs = driver.window_handles
            if len(tabs) > 1:
                # Cenário 2: Nova aba aberta
                driver.switch_to.window(tabs[1])

                # Localizar a tabela e encontrar a linha com "Registration Type" = 'full'
                try:
                    table = wait.until(EC.presence_of_element_located((By.XPATH, '//table')))
                    rows = table.find_elements(By.XPATH, './/tbody/tr')
                    found_row = False

                    for row in rows:
                        try:
                            registration_type = row.find_element(By.XPATH, './/td[contains(text(), "full")]')
                            if registration_type:
                                # Clicar no botão na mesma linha
                                button = row.find_element(By.XPATH, './/button')
                                button.click()
                                found_row = True
                                print(f"Linha com 'full' encontrada para o CAS {cas_number}. Botão clicado.")
                                break
                        except:
                            continue

                    if not found_row:
                        print(f"Nenhuma linha com 'full' encontrada para o CAS {cas_number}.")
                 except:
                    print(f"Tabela não encontrada na nova aba para o CAS {cas_number}.")

                # Capturar a URL da nova aba
                url_new_tab = driver.current_url
                print(f"URL da nova aba para o CAS {cas_number}: {url_new_tab}")

                # Fechar a nova aba e voltar para a aba original
                driver.close()
                driver.switch_to.window(tabs[0])
             else:
                # Cenário 1: Não abre nova aba
                url_current_page = driver.current_url
                print(f"URL da página de interesse para o CAS {cas_number}: {url_current_page}")

        except Exception as e:
            print(f"Erro ao processar o CAS Number {cas_number}: {e}")
            
            # Função auxiliar para buscar um elemento de forma segura
            def get_text_or_default(xpath, default="Não encontrado"):
                elements = driver.find_elements(By.XPATH, xpath)
                return elements[0].text if elements else default
                    
        except Exception as e:
            print(f"Erro ao processar o CAS Number {cas_number}: {e}")
        finally:
            # Fechar o navegador
            #driver.quit()
            pass
    return 'ok'

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 94)

In [48]:
echa_extractor("15663-27-1") # não existe o campo habilitado

Erro ao processar o CAS Number 15663-27-1: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=125.0.6422.141)
Stacktrace:
#0 0x5b8977047e3a <unknown>
#1 0x5b8976d3145c <unknown>
#2 0x5b8976d06a84 <unknown>
#3 0x5b8976da895d <unknown>
#4 0x5b8976dbec76 <unknown>
#5 0x5b8976da0253 <unknown>
#6 0x5b8976d701c7 <unknown>
#7 0x5b8976d70b3e <unknown>
#8 0x5b897700e27b <unknown>
#9 0x5b8977012327 <unknown>
#10 0x5b8976ffadae <unknown>
#11 0x5b8977012df2 <unknown>
#12 0x5b8976fdf74f <unknown>
#13 0x5b8977037128 <unknown>
#14 0x5b89770372fb <unknown>
#15 0x5b8977046f6c <unknown>
#16 0x7ecb8c694ac3 <unknown>



'ok'

In [39]:
echa_extractor("50-00-0") # existe mais de um 

Botão clicado. Nova aba aberta.
URL da nova aba: https://echa.europa.eu/pt/information-on-chemicals/registered-substances/-/disreg/substance/100.000.002


'ok'

In [ ]:
echa_extractor("13463-67-7") # existe um 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import chromedriver_autoinstaller
import time
import json

def echa_extractor(cas_numbers):
    # Certificar-se de que cas_numbers é uma lista
    if isinstance(cas_numbers, str):
        cas_numbers = [cas_numbers]

    # Instalar e configurar o driver automaticamente
    chromedriver_autoinstaller.install()
 
    for cas_number in cas_numbers:
        chrome_options = webdriver.ChromeOptions()
        driver = webdriver.Chrome(options=chrome_options)
        try:
            # Acessar a página
            driver.get("https://echa.europa.eu/pt/information-on-chemicals")

            # Configurar WebDriverWait
            wait = WebDriverWait(driver, 10)
            actions = ActionChains(driver)

            # Aceitar cookies
            cookie_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="cookie-consent-banner"]/div/div/div[2]/a[1]')))
            actions.move_to_element(cookie_button).click().perform()

            time.sleep(2)

            # Selecionar checkbox
            checkbox = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="_disssimplesearchhomepage_WAR_disssearchportlet_fm"]/div[2]/label/span')))
            actions.move_to_element(checkbox).click().perform()

            time.sleep(2)

            # Inserir o número CAS na barra de pesquisa
            search = driver.find_element(By.XPATH, '//*[@id="autocompleteKeywordInput"]')
            search.send_keys(cas_number)
            search.send_keys(Keys.RETURN)

            # Clicar no botão da toxina
            toxin_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="_disssimplesearch_WAR_disssearchportlet_rmlSearchResultVOsSearchContainerSearchContainer"]/table/tbody/tr[1]/td[1]/a')))
            toxin_button.click()

            # Verificar se o botão "Registered" está habilitado
            try:
                registered_button = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="_disssubsinfo_WAR_disssubsinfoportlet_dataset-reg-button"]/span')))
                if not registered_button.is_enabled():
                    print(f"Botão 'Registered' desabilitado para o CAS {cas_number}.")
                    continue  # Cenário 3: Botão desabilitado
            except:
                print(f"Botão 'Registered' não encontrado para o CAS {cas_number}.")
                continue

            # Se o botão "Registered" está habilitado, verificar se abre uma nova aba
            registered_button.click()
            time.sleep(2)

            # Verificar se uma nova aba foi aberta
            tabs = driver.window_handles
            if len(tabs) > 1:
                # Cenário 2: Nova aba aberta
                driver.switch_to.window(tabs[1])

                registered_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="_dissregisteredsubstances_WAR_dissregsubsportlet_disregOutputitemVOsSearchContainerSearchContainer"]/table/tbody/tr[1]/td[9]/a')))
                registered_button.click()
              
                    # Espera um pouco para a nova aba carregar (ou use WebDriverWait)
                time.sleep(2)
                    
                    # Alterna para a nova aba
                driver.switch_to.window(driver.window_handles[1])
                    
                # Capturar a URL da nova aba
                url_new_tab = driver.current_url
                print(f"URL da nova aba para o CAS {cas_number}: {url_new_tab}")

                if len(driver.window_handles) > 1:
                    for handle in driver.window_handles[:-1]:
                        driver.switch_to.window(handle)
                        driver.close()
                
                    # Alterna para a aba mais recente
                    driver.switch_to.window(driver.window_handles[0])

                # Fechar a nova aba e voltar para a aba original
                #driver.close()
                #driver.switch_to.window(tabs[0])
            else:
                # Cenário 1: Não abre nova aba
                url_current_page = driver.current_url
                print(f"URL da página de interesse para o CAS {cas_number}: {url_current_page}")


            nav_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="MainNav7"]/a')))
            nav_button.click()

            element = driver.find_element(By.XPATH, '//*[@id="SubNav7_1"]/a')
            driver.execute_script("arguments[0].click();", element)
    
            # XPath base
            xpath_base = '//*[@id="MainContent"]'
            
            # Encontrar elementos dentro da hierarquia do XPath base
            main_div = driver.find_element(By.XPATH, f"{xpath_base}//*")
                        
            # Encontra todas as seções <h3> dentro de main-content
            secoes_h3 = main_div.find_elements(By.TAG_NAME, "h3")
        
            # Lista para armazenar os dados organizados
            todas_secoes = []
        
            for secao_h3 in secoes_h3:
                secao_nome = secao_h3.text.strip()
                secao_dict = {"seção": secao_nome, "subseções": []}
        
                # Pegar todos os <h4> que aparecem depois desse <h3> e antes do próximo <h3>
                subsecoes_h4 = secao_h3.find_elements(By.XPATH, "./following-sibling::h4")
        
                for subsecao_h4 in subsecoes_h4:
                    subsecao_nome = subsecao_h4.text.strip()
                    subsecao_dict = {"subseção": subsecao_nome, "dados": {}}
        
                    # Pegar todos os <h5> que aparecem depois desse <h4> e antes do próximo <h4> ou <h3>
                    subsecoes_h5 = subsecao_h4.find_elements(By.XPATH, "./following-sibling::h5")
        
                    for subsecao_h5 in subsecoes_h5:
                        subsubsecao_nome = subsecao_h5.text.strip()
                        dados_dict = {"dados_textuais": []}
        
                        # Encontrar <dt> e <dd> dentro da mesma subseção
                        elementos_dt = subsecao_h5.find_elements(By.XPATH, "./following-sibling::dl/dt")
                        elementos_dd = subsecao_h5.find_elements(By.XPATH, "./following-sibling::dl/dd")
        
                        for dt, dd in zip(elementos_dt, elementos_dd):
                            chave = dt.text.strip()
                            valor = dd.text.strip()
                            dados_dict[chave] = valor
        
                        # Capturar parágrafos <p> dentro da subseção
                        paragrafos = subsecao_h5.find_elements(By.XPATH, "./following-sibling::p")
                        for p in paragrafos:
                            texto_p = p.text.strip()
                            if texto_p:  # Evita adicionar parágrafos vazios
                                dados_dict["dados_textuais"].append(texto_p)
        
                        subsecao_dict["dados"][subsubsecao_nome] = dados_dict
        
                    secao_dict["subseções"].append(subsecao_dict)
        
                todas_secoes.append(secao_dict)
        
            # Converte para JSON formatado
            json_resultado = json.dumps(todas_secoes, indent=4, ensure_ascii=False)
        
            print(json_resultado)

        except Exception as e:
            print(f"Erro ao processar o CAS Number {cas_number}: {e}")
        finally:
            # Fechar o navegador
            driver.quit()
            pass

    return json_resultado  # Retorna o DataFrame com os dados coletados (se houver)

In [38]:
echa_extractor("50-00-0")

URL da nova aba para o CAS 50-00-0: https://echa.europa.eu/pt/information-on-chemicals/registered-substances/-/disreg/substance/100.000.002
[
    {
        "seção": "Administrative data",
        "subseções": [
            {
                "subseção": "Systemic effects",
                "dados": {
                    "Long term exposure": {
                        "dados_textuais": [],
                        "Hazard assessment conclusion:": "medium hazard (no threshold derived)",
                        "Value:": "82 mg/kg bw/day",
                        "Most sensitive endpoint:": "repeated dose toxicity",
                        "Route of original study:": "Oral",
                        "DNEL derivation method:": "ECHA REACH Guidance",
                        "Overall assessment factor (AF):": "20",
                        "Dose descriptor starting point:": "NOAEL",
                        "Modified dose descriptor starting point:": "NOAEL",
                        "Explanation f

'ok'